In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as pltx

2025-09-11 17:56:33.187906: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-09-11 17:56:33.277558: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-09-11 17:56:34.998706: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
# Define custom LogNeuron
class LogNeuron(tf.keras.Model):
    def __init__(self):
        super().__init__()
        # weight and bias initialized randomly like in your PyTorch version
        self.weight = tf.Variable(tf.random.uniform([1], minval=0.5, maxval=1.5), trainable=True)
        self.bias = tf.Variable(tf.random.uniform([1], minval=0.5, maxval=1.5), trainable=True)

    def call(self, x):
        return tf.math.log(self.weight * x + self.bias)

In [3]:
# Training data
x = tf.constant([0.001, 0.01, 1.0, 2.0, 3.0, 4.0], dtype=tf.float32)
y_target = tf.math.log(x + 1.0)

2025-09-11 17:56:36.321654: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


In [4]:
# Create model
neuron = LogNeuron()

# Loss function (MSE)
loss_fn = tf.keras.losses.MeanSquaredError()

# Optimizer (SGD)
optimizer = tf.keras.optimizers.SGD(learning_rate=0.01)

# Training loop
epochs = 10000
outputs_history = np.zeros((epochs, len(x)))

for epoch in range(epochs):
    with tf.GradientTape() as tape:
        output = neuron(x)
        loss = loss_fn(y_target, output)
    # Compute gradients
    grads = tape.gradient(loss, [neuron.weight, neuron.bias])
    # Apply gradients
    optimizer.apply_gradients(zip(grads, [neuron.weight, neuron.bias]))
    # Store history
    outputs_history[epoch, :] = output.numpy()

print("Trained weight:", neuron.weight.numpy()[0])
print("Trained bias:", neuron.bias.numpy()[0])

Trained weight: 0.99999076
Trained bias: 1.0000105


In [8]:
# Test inputs
output = neuron(x)
# True target outputs for comparison
y_true = tf.math.log(x_test + 1)
# Let Neuron calulcate values
y_pred = neuron(x_test)

# Print comparison with deviation and percentsage error
print("x\tNeuron output\tTarget output\tDeviation\t% Error")
for xi, yp, yt in zip(x_test, y_pred, y_true):
    deviation = yp - yt
    percent_error = abs(deviation / yt) * 100
    print(f"{xi.item():.2f}\t{yp.item():.4f}\t\t{yt.item():.4f}\t\t{deviation.item():.4f}\t\t{percent_error.item():.2f}%")


NameError: name 'x_test' is not defined